This notebook will introduce you to `kcgof` (kernel conditional goodness-of-fit testing), a Python package implementing two kernel-based conditional goodness-of-fit tests as described in

    Testing Goodness of Fit of Conditional Density Models with Kernels
    Wittawat Jitkrittum, Heishiro Kanagawa, Bernhard Schölkopf
    UAI 2020
    https://arxiv.org/abs/2002.10271

See [the Github page](https://github.com/wittawatj/kernel-cgof) for more information.

Make sure that you have `kcgof` included in Python's search path. In particular the following import statements should not produce any fatal error. For how to install software dependencies, see the [README](https://github.com/wittawatj/kernel-cgof). In our code, we use [Pytorch](https://pytorch.org/).

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
import torch

import kcgof
import kcgof.log as klog
import kcgof.util as util
import kcgof.cdensity as cden
import kcgof.cdata as cdat
import kcgof.cgoftest as cgof
import kcgof.kernel as ker
import kcgof.plot as plot

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

# font options
font = {
    #'family' : 'normal',
    #'weight' : 'bold',
    'size'   : 20
}

plt.rc('font', **font)
plt.rc('lines', linewidth=2)
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

## Conditional Goodness-of-fit testing

Given a known conditional density model $p(y|x)$ and a sample $\{ (x_i, y_i) \}_{i=1}^n \sim q(y|x) r(x)$ where $q$ is the true conditional distribution generating the data, and $r$ is the marginal distribution for $x$, a conditional goodness-of-fit test proposes the following null hypothesis

$H_0: p(y|x) = q(y|x) \, \text{for almost all}\, x,y$ 

against the alternative hypothesis $H_1$ which is simply the negation of $H_0$.

In other words, it tests whether or not the sample follows the model $p(y|x)$, for some marginal distribution $r$. Note that the model does not specify the marginal distribution.

## Our work

We propose two new tests: 

1. The Kernel Conditional Stein Discrepancy (KCSD)
2. The Finite Set Conditional Discrepancy (FSCD)

The KCSD test makes a few mild assumptions on the distributions $p$ and $q$. 
The model $p(y|x)$ can take almost any form as long as it is differentiable. 
The normalizer of $p(y|x)$ (i.e., $p(x)$) is not assumed known. 
The test only assesses the goodness of the model through $\nabla_{y} \log p(y|x)$ i.e., the first derivative of the log conditional density. The FSCD is an extention of KCSD to return an interpretable set of locations in the space of $x$ that indicate where $p(y|x)$ differs from $q(y|x)$.


### Specifying your model

In our code, a model is represented as an instance of `kcgof.cdensity.UnnormalizedCondModel`. You can subclass this class and implement your model. Alternatively, you can represent your model by the following convenient method that will construct an instance of this class for you.

        kcgof.cdensity.from_log_den(dx, dy, f)
        
where:
* `dx` is the dimension of x (positive integer)
* `dy` is the dimension of y (positive integer)
* `f` is a callable (a function) that takes two arguments (X, Y), where X: n x dx, Y: n x dy Torch tensors, and evaluates the unnormalized conditional density of the model i.e., p(x,y) (up to any mutiplicative factor as that is a function of x). Return a Torch array of length n.
    

## Ordinary least squares with Gaussian noise


For illustration, let us consider a simple one-dimensional ($d_x=1, d_y=1$) toy problem. Suppose our model is

$$p(y|x) = \mathcal{N}(y \mid slope \cdot x+c, variance),$$

for some "slope", "c", and "variance". This is the standard least squares problem with Gaussian noise. 
In general, our tests can be applied to $x \in \mathbb{R}^{d_x}$ and $\mathbb{R}^{d_y}$ where $d_x \ge 1$ and $d_y \ge 1$.

In this case, to specify the model, we first implement a function that computes the log density of the model, up to the normalizer. In this case, that would be 

$$ - \frac{(y - (slope\cdot x + c))^2}{2 \times variance}.$$

Note that the normalizer of the Gaussian density is dropped here. In general, any multiplicative factor on the density function that is independent of $y$ can be dropped. The reason is that our tests only use $\nabla_y \log p(y|x) = \nabla_y \log p(x, y)$. So any multiplicative factor will be irrelevant once the derivative $\nabla_y$ is computed. The derivative will be computed automatically.

In [ ]:
# First define a function that implements the log density up to the normalizer
def simple_gaussian_model(X, Y):    
    # Some arbitrary model parameters here only for illustration.
    # In practice, any models parameters are likely learned from 
    # an independent set of data before testing.
    slope = torch.tensor([1.0])
    c = 1.0
    variance = 1.0
    
    # Note that X (n x dx), Y (n x dy) are Torch tensors
    linear = torch.matmul(X, slope) + c
    
    # Note that this normalizing constant is not necessary. 
    # Try to set it to zero later to see it for yourself that the test result is not changed.
    constant = -np.log(variance**0.5) - (2.0*np.pi)**0.5
#     constant = 0
    return -(Y.view(-1)-linear)**2/(2.0*variance) + constant

In [ ]:
dx = 1
dy = 1
# Model p is an instance of kcgof.cdensity.UnnormalizedCondModel 
# and is ready for conditional goodness-of-fit testing.
p = cden.from_log_den(dx, dy, simple_gaussian_model)

### A toy problem

In [ ]:
# dimension of x. 
dx = 1

To illustrate the test, let us generate some toy data.

In [ ]:
# import torch.distributions as dists
np.random.seed(10)
real_slope = torch.tensor([1.3])
real_variance = 1.0
real_c = torch.tensor([1.0])

# sample size
n = 500 
X = torch.tensor(np.random.randn(n, dx)*2.0, dtype=torch.float32)
linear = torch.matmul(X, real_slope) + real_c
output_noise = np.random.randn(n)*real_variance**0.5
Y = linear + output_noise
Y = Y.view(n, 1).type(torch.float32)

Plot the data and the model

In [ ]:
def plot_2d_data_model(p, X, Y, domX, domY, figsize=(10, 6), levels=20,
    cmap='pink_r', **contourop ):
    """
    Plot the conditional density model p(y|x) along with the data on a 2d plot.
    Both x, and y must be scalar-valued. 
    p: cdensity.UnnormalizedCondDensity object representing the model p(y|x)
    X, Y: n x 1 torch tensors for the data of x and y
    domX: n x 1 torch tensor specifying points to plot in the domain of x
    domY: n x 1 torch tensor specifying points to plot in the domain of y
    
    """
    mdomX, mdomY = torch.meshgrid(domX.view(-1), domY.view(-1))
    flatlogden = p.log_den(mdomX.reshape(-1, 1), mdomY.reshape(-1, 1))
    flatden = torch.exp(flatlogden)
    # for the purpose of plotting, if the density is Nan, make it 0
    flatden[torch.isnan(flatden)] = 0.0
    mden = flatden.view(mdomX.shape)

    np_mdomX = mdomX.detach().numpy()
    np_mdomY = mdomY.detach().numpy()
    np_mden = mden.detach().numpy()

    plt.contourf(np_mdomX, np_mdomY, np_mden, levels=levels, cmap=cmap,
        **contourop)
    npX = X.detach().numpy()
    npY = Y.detach().numpy()
    plt.plot(npX, npY, 'bo', markersize=3)
    plt.ylabel('$p(y|x)$')
    # plt.colorbar()

In [ ]:
ep = 0.7
# make a grid that covers X
domX = torch.linspace(torch.min(X)-ep, torch.max(X)+ep, 100)
domY = torch.linspace(torch.min(Y).item()-ep, torch.max(Y).item()+ep, 200)

plot_2d_data_model(p, X, Y, domX, domY)

Note that our model is slightly wrong since it has a wrong slope.

### The Kernel Conditional Stein Discrepancy (KCSD) Test

In [ ]:
# Use Gaussian kernels for both X and Y.
# Let us use the median heuristic to pick the bandwidths.
sigx = util.pt_meddistance(X, subsample=1000, seed=2)
sigy = util.pt_meddistance(Y, subsample=1000, seed=3)
        
# k = kernel on X
k = ker.PTKGauss(sigma2=sigx**2)

# l = kernel on Y
l = ker.PTKGauss(sigma2=sigy**2)

In [ ]:
# Construct a KCSD test object
# alpha = significant level for the test
# n_bootstrap = the number of times to run wild bootstrap to simulate 
# the null distribution. The larger, the better.
# The model p is used here to construct the test.
kcsdtest = cgof.KCSDTest(p, k, l, alpha=0.05, n_bootstrap=500)

In [ ]:
# perform the actual test on the data 
result = kcsdtest.perform_test(X, Y)
result

It can be seen that the test rejects $H_0$ since the model is wrong. Go back and change the slope of the model to match the data generating process. With probability roughly $1-\alpha$, the test will not reject.

## Optimized KCSD test

The KCSD test relies on two kernels $k$ and $l$ for $x$ and $y$ respectively. 
So far, we use Gaussian kernels and set the bandwidths by the median heuristic. 
A more principled way to choose the bandwidths is by optimizing them so as to 
maximize the asymptotic test power. If $H_1$ is true, optimizing the kernels
in this way can lead to higher test power. If $H_0$ is true, the false rejection 
rate of $H_0$ is still under control (i.e., asymptotically no larger than $\alpha$).

For this particular toy problem that we consider, however, optimizing the two kernels
may not be necessary since the problem is quite simple.

In [ ]:
# Split the data into training set and test set.
# The training set will be used to tune the two bandwidths.
# Splitting ensures that the type-I error (false rejection rate)
# is well controlled.

# Proportion of the training set. The rest are for testing.
tr_proportion = 0.3
tr, te = cdat.CondData(X, Y).split_tr_te(tr_proportion=tr_proportion)

# Xtr, Ytr are subset of the original data (X,Y)
Xtr, Ytr = tr.xy()
print(Xtr.shape)

In [ ]:
Ytr.requires_grad = False
kcsd_pc = cgof.KCSDPowerCriterion(p, k, l, Xtr, Ytr)

max_iter = 200

# learning rate 
lr = 1e-3
# regularization
reg = 1e-3

# constraint satisfaction function
def con_f(params):
    ksigma2 = params[0]
    lsigma2 = params[1]
    ksigma2.data.clamp_(min=1e-2, max=10)
    lsigma2.data.clamp_(min=1e-2, max=10)
    
objs = kcsd_pc.optimize_params(
    [k.sigma2, l.sigma2], constraint_f=con_f,
    lr=lr, reg=reg, max_iter=max_iter)

In [ ]:
np_objs = objs.detach().numpy()

plt.figure(figsize=(8,5))
plt.plot(np.arange(max_iter), np_objs, 'b-')
plt.xlabel('iteration')
plt.ylabel('Power criterion')

In [ ]:
k.sigma2

In [ ]:
l.sigma2

Test on the test set

In [ ]:
# Construct a KCSD test object
kcsdtest = cgof.KCSDTest(p, k, l, alpha=0.05, n_bootstrap=400, seed=9)
Xte, Yte = te.xy()
kcsdtest.perform_test(Xte, Yte)